In [ ]:
%load_ext autoreload
%autoreload 2

# %set_env CUDA_VISIBLE_DEVICES=7
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BootstrapFinetune

### 1) 配置默认的语言模型和检索器

In [2]:
# 定义端口列表
ports = [7140, 7141, 7142, 7143, 7144, 7145]
# 创建一个 HFClientTGI 对象 llamaChat，使用指定的模型和端口，最大 token 数为 150
llamaChat = dspy.HFClientTGI(model="meta-llama/Llama-2-13b-chat-hf", port=ports, max_tokens=150)
# 创建一个 ColBERTv2 对象 colbertv2，指定 URL 地址
colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# 配置 dspy 库的设置，指定 rm 为 colbertv2，lm 为 llamaChat
dspy.settings.configure(rm=colbertv2, lm=llamaChat)

### 2) 加载 HotPotQA 数据的一个小样本

In [3]:
# 导入HotPotQA数据集
dataset = HotPotQA(train_seed=1, train_size=200, eval_seed=2023, dev_size=1000, test_size=0)

# 从训练集中提取问题作为输入
trainset = [x.with_inputs('question') for x in dataset.train]

# 从开发集中提取问题作为输入
devset = [x.with_inputs('question') for x in dataset.dev]

# 从测试集中提取问题作为输入
testset = [x.with_inputs('question') for x in dataset.test]

# 输出训练集、开发集和测试集的长度
len(trainset), len(devset), len(testset)

(200, 1000, 0)

In [21]:
# 输出训练集中的第一个样本
trainset[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

### 3) 定义一个简单的多跳程序

In [4]:
from dsp.utils.utils import deduplicate

# 定义一个类 BasicMH，继承自 dspy.Module
class BasicMH(dspy.Module):
    def __init__(self, passages_per_hop=3):
        super().__init__()

        # 初始化 Retrieve 模块，设置参数 k 为 passages_per_hop
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        # 初始化两个 ChainOfThought 模块，用于生成查询语句
        self.generate_query = [dspy.ChainOfThought("context, question -> search_query") for _ in range(2)]
        # 初始化 ChainOfThought 模块，用于生成答案
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = []
        
        # 进行两轮循环，每轮循环生成查询语句并获取相关文本，更新上下文
        for hop in range(2):
            search_query = self.generate_query[hop](context=context, question=question).search_query
            passages = self.retrieve(search_query).passages
            context = deduplicate(context + passages)

        # 生成最终答案并返回
        return self.generate_answer(context=context, question=question).copy(context=context)

### 4) 使用 `Llama2-13b-chat` 编译该程序

In [5]:
# 是否重新从头开始编译成 LLAMA
RECOMPILE_INTO_LLAMA_FROM_SCRATCH = False

# 线程数
NUM_THREADS = 24

# 使用精确匹配作为度量标准
metric_EM = dspy.evaluate.answer_exact_match

In [6]:
# 如果需要从头开始重新编译成LLAMA
if RECOMPILE_INTO_LLAMA_FROM_SCRATCH:
    # 使用BootstrapFewShotWithRandomSearch类进行初始化
    tp = BootstrapFewShotWithRandomSearch(metric=metric_EM, max_bootstrapped_demos=2, num_threads=NUM_THREADS)
    # 编译BasicMH模型，使用前50个数据进行训练，50到200个数据作为验证集
    basicmh_bs = tp.compile(BasicMH(), trainset=trainset[:50], valset=trainset[50:200])

    # 从候选程序中选择前4个程序组成集合ensemble
    ensemble = [prog for *_, prog in basicmh_bs.candidate_programs[:4]]

    # 遍历ensemble集合中的程序，并保存到对应的文件中
    for idx, prog in enumerate(ensemble):
        # prog.save(f'checkpoints/multihop_llama213b_{idx}.json')
        pass

In [7]:
# 如果不需要从头开始重新编译LLAMA，则执行以下代码
if not RECOMPILE_INTO_LLAMA_FROM_SCRATCH:
    ensemble = []  # 创建一个空列表用于存储模型

    for idx in range(4):
        prog = BasicMH()  # 创建一个BasicMH对象
        prog.load(f'checkpoints/multihop_llama213b_{idx}.json')  # 从指定路径加载模型参数
        ensemble.append(prog)  # 将加载的模型添加到ensemble列表中

In [8]:
# 选择第一个模型作为llama_program
llama_program = ensemble[0]

# 创建一个Evaluate对象，用于评估模型在开发集上的表现
# 参数包括开发集的前1000个样本，评估指标为EM（精确匹配），线程数为NUM_THREADS，显示进度为True，不显示表格
evaluate_hotpot = Evaluate(devset=devset[:1000], metric=metric_EM, num_threads=NUM_THREADS, display_progress=True, display_table=0)

# 对llama_program进行评估
evaluate_hotpot(llama_program)

Average Metric: 424 / 1000  (42.4): 100%|██████████| 1000/1000 [00:14<00:00, 70.51it/s]


Average Metric: 424 / 1000  (42.4%)


42.4

In [ ]:
# 调用llama_program函数，参数为问题字符串"David Gregory继承的城堡有多少层楼？"
llama_program(question="David Gregory继承的城堡有多少层楼？")

# 调用llamaChat对象的inspect_history方法，参数n为3，表示查看最近的3条对话记录
llamaChat.inspect_history(n=3)

### 6) 编译成 `T5-Large`（770M 参数）

In [10]:


# 创建一个HotPotQA数据集实例，用于训练集
unlabeled_train = HotPotQA(train_seed=1, train_size=3000, eval_seed=2023, dev_size=0, test_size=0).train

# 将训练集中的问题提取出来，构建成新的数据集
unlabeled_train = [dspy.Example(question=x.question).with_inputs('question') for x in unlabeled_train]

# 输出新数据集的长度
len(unlabeled_train)

3000

可选步骤：在未标记的训练集上预先计算集成模型

In [ ]:
# 定义一个始终返回True的lambda函数，接受三个参数g, p, trace，其中trace参数默认为None
always_true = lambda g, p, trace=None: True

# 遍历ensemble列表中的每个元素
for prog_ in ensemble:
    # 对prog_调用evaluate_hotpot函数，传入参数devset=unlabeled_train[:3000]和metric=always_true
    evaluate_hotpot(prog_, devset=unlabeled_train[:3000], metric=always_true)

现在编译成T5！

In [13]:
RECOMPILE_INTO_T5_FROM_SCRATCH = False

if RECOMPILE_INTO_T5_FROM_SCRATCH:
    config = dict(target='t5-large', epochs=2, bf16=True, bsize=6, accumsteps=2, lr=5e-5)

    tp = BootstrapFinetune(metric=None)
    t5_program = tp.compile(BasicMH(), teacher=ensemble, trainset=unlabeled_train[:3000], **config)

    # 关闭思维链提示。让我们使用T5直接预测输出。（更快且质量相似。）
    for p in t5_program.predictors(): p.activated = False

In [ ]:
if not RECOMPILE_INTO_T5_FROM_SCRATCH:
    t5_program = BasicMH()

    # ckpt_path = '../finetuning_ckpts/LMWEP0WZ5IKWM.all/checkpoint-5400'
    ckpt_path = "colbert-ir/dspy-Oct11-T5-Large-MH-3k-v1"
    LM = dspy.HFModel(checkpoint=ckpt_path, model='t5-large')

    for p in t5_program.predictors():
        p.lm = LM
        p.activated = False

### 7) 评估 T5-Large `multihop` 程序

In [ ]:
# 调用 evaluate_hotpot 函数计算分数，并将结果赋值给 score
score = evaluate_hotpot(t5_program, num_threads=1)

In [ ]:
# 获取 T5 模型的第一个预测器，并调用 inspect_history 方法查看历史记录
t5_program.predictors()[0].lm.inspect_history(n=3)